In [7]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import datetime
import os
import sys
import nanonis_importer

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

SLABS_TYPES = []
for slab in CONFIG["slabs"]:
    object_type = DATA_MODEL["classes"][slab]["annotations"]["openbis_label"].replace(" ", "_").upper()
    SLABS_TYPES.append(object_type)

SAMPLES_COLLECTION_OPENBIS_PATH = CONFIG["samples_collection_openbis_path"]
MEASUREMENT_FILE_EXTENSIONS = CONFIG["measurement_file_extensions"]

experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

sample_selector_config = {
    "dropdown": {"width": "315px"},
    "details": {"width": "589px", "height": "250px"}
}
sample_selector = widgets.ObjectSelectionWidget("Sample", sample_selector_config, contains_label = False)
sample_selector.load_dropdown_box()

instrument_selector = widgets.ObjectSelectionWidget("Instrument", contains_label = False)
instrument_selector.load_dropdown_box()

sample_out_name_textbox = utils.Text(
    description = "Name", disabled = False, layout = ipw.Layout(width = '400px'), 
    placeholder = f"Write sample name here...", style = {'description_width': "110px"}
)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)

save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

folder_selector = utils.FileChooser(path = '.', select_default=True, use_dir_icons=True, show_only_dirs = True)

In [ ]:
def get_sample_processes(sample_identifier):
    objects_database = {}
    sample_processes = {}
    sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_identifier)
    objects_stack = [(sample_object, None)]
    
    while objects_stack:
        current_object, child_object = objects_stack.pop()
        if current_object.identifier in objects_database:
            current_object_data = objects_database[current_object.identifier][1]
        else:
            current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid, DATA_MODEL)
            objects_database[current_object.identifier] = [current_object, current_object_data]
        
        if current_object_data["type"] == "PROCESS_STEP" and current_object_data["permId"] not in sample_processes:
            process_permid = current_object_data["permId"]
            process_instrument = current_object_data["props"]["instrument"]
            
            process_actions = current_object_data["props"]["actions"]
            if process_actions is None:
                process_actions = []
                
            process_observables = current_object_data["props"]["observables"]
            if process_observables is None:
                process_observables = []
                
            process_children = current_object_data["children"]
            for child_identifier in process_children:
                if child_identifier not in objects_database:
                    child_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = child_identifier)
                    child_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, child_identifier, DATA_MODEL)
                    objects_database[child_identifier] = [child_object, child_object_data]
                else:
                    child_object_data = objects_database[child_identifier][1]
            
            if process_actions or process_observables:   
                sample_processes[process_permid] = {
                    "actions": process_actions, 
                    "observables": process_observables,
                    "instrument": process_instrument
                }
        
        for parent_identifier in current_object.parents:
            if parent_identifier not in objects_database:
                parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
                parent_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, parent_identifier, DATA_MODEL)
                objects_database[parent_identifier] = [parent_object, parent_object_data]
            else:
                parent_object = objects_database[parent_identifier][0]
                
            objects_stack.append((parent_object, current_object))
            
    return sample_processes

def load_sample_metadata(change):
    if sample_selector.dropdown.value == -1:
        instrument_selector.dropdown.value = -1
        sample_selector.details_textbox.value = ''
        sample_out_name_textbox.value = ''
        return
    else:
        sample_processes = get_sample_processes(sample_selector.dropdown.value)
        
        last_sample_process = None
        for process_identifier in sample_processes:
            current_process = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = process_identifier)
            if last_sample_process:
                if last_sample_process.registrationDate < current_process.registrationDate:
                    last_sample_process = current_process
            else:
                last_sample_process = current_process
        
        if last_sample_process:
            last_sample_experiment = last_sample_process.experiment
            # Notify the user in case the experiment changed
            if experiment_selector.dropdown.value != last_sample_experiment.permId and experiment_selector.dropdown.value != -1:
                dropdown_options_dict = {key: val for val, key in experiment_selector.dropdown.options}
                previous_experiment_name = dropdown_options_dict.get(experiment_selector.dropdown.value)
                new_experiment_name = dropdown_options_dict.get(last_sample_experiment.permId)
                display(utils.Javascript(data = f"alert('{f'Experiment was changed from {previous_experiment_name} to {new_experiment_name}!'}')"))
                    
            experiment_selector.dropdown.value = last_sample_experiment.permId

# def load_sample_metadata(change):
#     if sample_selector.dropdown.value == -1:
#         instrument_selector.dropdown.value = -1
#         sample_selector.details_textbox.value = ''
#         sample_out_name_textbox.value = ''
#         return
    
#     sample_object = OPENBIS_SESSION.get_object(sample_selector.dropdown.value)
#     sample_details = utils.get_sample_details(OPENBIS_SESSION, sample_object, SAMPLE_PREPARATION_SAMPLE_TYPES, SLABS_TYPES)
    
#     sample_metadata_string = (f"PermId: {sample_object.attrs.permId}\nMaterial:\n" +
#                             "\n".join(sample_details["materials"]) + 
#                             "\nProcesses:\n" + 
#                             "\n".join(sample_details["sample_preparations"]))
    
#     last_sample_preparation = sample_details["last_sample_preparation_object"]
    
#     if last_sample_preparation:
#         last_sample_preparation_object = OPENBIS_SESSION.get_object(last_sample_preparation)
#         last_sample_experiment = utils.get_last_sample_experiment(OPENBIS_SESSION, sample_object, last_sample_preparation_object)
        
#         # Notify the user in case the experiment changed
#         if experiment_selector.dropdown.value != last_sample_experiment.permId and experiment_selector.dropdown.value != -1:
#             dropdown_options_dict = {key: val for val, key in experiment_selector.dropdown.options}
#             previous_experiment_name = dropdown_options_dict.get(experiment_selector.dropdown.value)
#             new_experiment_name = dropdown_options_dict.get(last_sample_experiment.permId)
#             display(utils.Javascript(data = f"alert('{f'Experiment was changed from {previous_experiment_name} to {new_experiment_name}!'}')"))

#         instrument_object = utils.get_last_sample_instrument(OPENBIS_SESSION, last_sample_preparation_object)
        
#         experiment_selector.dropdown.value = last_sample_experiment.permId
#         instrument_selector.dropdown.value = instrument_object.permId
    
#     sample_selector.details_textbox.value = sample_metadata_string
#     sample_out_name = sample_object.props['name']
#     sample_out_name_textbox.value = sample_out_name

def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))
    
def create_measurements(folderpath, sample_experiment, sample_object, object):
    instrument_identifier = instrument_selector.dropdown.value
    paths = utils.full_listdir(folderpath)
    any_readable_measurement = any(f".{filename.split('.')[-1]}" in MEASUREMENT_FILE_EXTENSIONS for filename in paths)
    
    if any_readable_measurement:
        object.parents = [sample_object.permId, instrument_identifier]
        object.save()
        nanonis_importer.upload_measurements_into_openbis(
                SESSION_DATA['url'], folderpath,
                sample_experiment, sample_object.permId,
                object.permId,
                instrument_selector.dropdown.value
        )
        
    else:
        output_message = f"Folder does not contain any readable measurement file: {', '.join(MEASUREMENT_FILE_EXTENSIONS)}."
        print(output_message)
        
    for path in paths:
        if os.path.isdir(path):
            path_exists_in_openbis = False
            object_children = object.get_children()
            for child in object_children:
                if child.type == "MEASUREMENT_SESSION" and child.props["name"] == path:
                    path_exists_in_openbis = True
                    break
            
            if path_exists_in_openbis:
                measurements_object = child
            else:
                measurements_object = utils.create_openbis_object(
                    OPENBIS_SESSION, type = "MEASUREMENT_SESSION", 
                    collection = sample_experiment, 
                    props = {
                        "name": path,
                        "default_object_view": "IMAGING_GALLERY_VIEW",
                        "measurements_folder_path": path
                    },
                    parents = [object.permId]
                )
                
            create_measurements(path, sample_experiment, sample_object, measurements_object)

def upload_measurements_to_openbis(b):
    sample_object = OPENBIS_SESSION.get_object(sample_selector.dropdown.value)
    
    # Get sample experiment from parent
    sample_experiment = experiment_selector.dropdown.value
    
    if not sample_experiment:
        print("Sample does not belong to any project yet.")
        return
    
    # Check if sample is being measured for the first time after the preparation or if
    # some measurements were already performed before the next preparation
    measurements_object = None
    new_measurements = True
    if sample_object.children:
        for sample_child in sample_object.children:
            sample_child_object = OPENBIS_SESSION.get_object(sample_child)
            if sample_child_object.type in ["MEASUREMENT_SESSION"]:
                measurements_object = utils.find_first_object_permid(OPENBIS_SESSION, sample_child_object, "MEASUREMENT_SESSION")
                # The same sample may be used in different experiments, e.g., sample with a crystal used for calibration
                if measurements_object.experiment.permId == sample_experiment:
                    new_measurements = False
                    break
    
    # Register measurements in a different experiment then the preparation experiment
    if measurements_object:
        if measurements_object.experiment.permId != sample_experiment:
            new_measurements = True
    
    data_folder = folder_selector.selected_path
    sample_name = sample_object.props["name"]
    if new_measurements:
        measurements_object = utils.create_openbis_object(
            OPENBIS_SESSION, type = "MEASUREMENT_SESSION", 
            collection = sample_experiment, 
            props = {
                "name": f"Measurements of {sample_name}",
                "default_object_view": "IMAGING_GALLERY_VIEW",
                "measurements_folder_path": data_folder
            }
        )
        
    create_measurements(data_folder, sample_experiment, sample_object, measurements_object)
    
    # Reset widgets
    experiment_selector.dropdown.value = -1
    instrument_selector.dropdown.value = -1
    sample_selector.dropdown.value = -1


# Measurement Uploader

## Select experiment

In [ ]:
display(experiment_selector)

## Select sample

In [ ]:
display(sample_selector)
sample_selector.dropdown.observe(load_sample_metadata, names = 'value')

## Select instrument

In [8]:
display(instrument_selector)

NameError: name 'instrument_selector' is not defined

## Select measurements folders

In [ ]:
display(folder_selector)
display(save_close_buttons_hbox)
display(increase_buttons_size)

In [ ]:
create_button.on_click(upload_measurements_to_openbis)
quit_button.on_click(close_notebook)

['/home/jovyan/prep1/RT008/01_06_RT008',
 '/home/jovyan/prep1/RT008/01_02_RT008',
 '/home/jovyan/prep1/RT008/01_04_RT008']